!pip install --upgrade tensorflow

!pip install matplotlib

import keras
print(keras.__version__)

!pip install --upgrade keras

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.layers import Dense, LSTM
import numpy as np


data_path ="/Users/mdebosz/Documents/Prywatne/Python-Scripts/GSTFI/src/data/data.csv"
# Wczytywanie danych
df = pd.read_csv(data_path)

2024-05-21 15:39:32.807574: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df.drop(['Unnamed: 0'],axis=1, inplace=True)

In [3]:
# Wizualizacja danych
#plt.figure(figsize=(16,8))
#plt.title('Historia cen akcji')
#plt.plot(df['Goldman Sachs Japonia (K)'])
#plt.xlabel('Data', fontsize=18)
#plt.ylabel('Cena zamknięcia', fontsize=18)
#plt.show()

# Przygotowanie danych
data = df.filter(['Goldman Sachs Japonia (K)'])
dataset = data.values
training_data_len = int(np.ceil( len(dataset) * .95 ))

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

train_data = scaled_data[0:int(training_data_len), :]
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# Budowanie modelu LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(x_train, y_train, batch_size=1, epochs=1)

# Tworzenie testowego zestawu danych
test_data = scaled_data[training_data_len - 60: , :]

x_test = []
y_test = dataset[training_data_len:, :]

for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    
x_test = np.array(x_test)

x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

# Przewidzenia cen na próbce testowej
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# Wizualizacja przewidywań
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

"""plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Data', fontsize=18)
plt.ylabel('Cena zamknięcia', fontsize=18)
plt.plot(train['Goldman Sachs Japonia (K)'])
plt.plot(valid[['Goldman Sachs Japonia (K)', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()"""

/Users/mdebosz/opt/anaconda3/envs/dash/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2348/2348 ━━━━━━━━━━━━━━━━━━━━ 38s 15ms/step - loss: 0.0017
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step


/var/folders/9z/wqfhpsqd2jzgb0xgj6l5y6p80000gn/T/ipykernel_9959/1527951265.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


"plt.figure(figsize=(16,8))\nplt.title('Model')\nplt.xlabel('Data', fontsize=18)\nplt.ylabel('Cena zamknięcia', fontsize=18)\nplt.plot(train['Goldman Sachs Japonia (K)'])\nplt.plot(valid[['Goldman Sachs Japonia (K)', 'Predictions']])\nplt.legend(['Train', 'Val', 'Predictions'], loc='lower right')\nplt.show()"

In [6]:
valid

,Goldman Sachs Japonia (K),Predictions
2408,322.46,327.480560
2409,325.26,327.694336
2410,323.69,327.729401
2411,323.93,327.059021
2412,327.08,326.294586
...,...,...
2529,386.37,387.705109
2530,390.50,388.495789
2531,386.59,389.813385
2532,388.34,389.632568
